In [1]:
import pandas as pd
from PSAP.Problem import decimal_to_time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# THE SAME AS THE MH MODEL
df_ml = pd.read_excel('../../results/TABU/ML-Results/ML151-200x10-redo-c.xlsx')
df_ml = df_ml.rename(columns={'number of movements': 'movements_reached-ML', 'median delay': 'median_delay-ML', 'average delay': 'average_delay-ML', 'obj_val': 'obj_val-ML', 'solution_found': 'valid_solution-ML', 'predicted_delay': 'predicted_delay-ML','tabu_list_size': 'tabu_list_size-ML', 'number_of_tweaks': 'number_of_tweaks-ML', 'affected_movements': 'affected_movements-ML',})

# add 100 to the instance number
df_ml['instance'] = df_ml['instance'] + 100
print(df_ml.head(20))

df_tabu = pd.read_excel('results/TABU/output_1000e_200_ctd1.xlsx')

df_tabu = df_tabu.rename(columns={'number of movements': 'movements_reached-TABU', 'median delay': 'median_delay-TABU', 'average delay': 'average_delay-TABU', 'obj_val': 'obj_val-TABU', 'solution_found': 'valid_solution-TABU', 'predicted_delay': 'predicted_delay-TABU', 'tabu_list_size': 'tabu_list_size-TABU', 'number_of_tweaks': 'number_of_tweaks-TABU', 'affected_movements': 'affected_movements-TABU',})

df_tabu = df_tabu[1500:3000]

# drop the rows that are not needed
# keep the ones that are divisible by 3
df_tabu = df_tabu[df_tabu.index % 3 == 0]


# load in the instance data
df_instances = pd.read_excel('results/instanceData_200.xlsx')
df_instances = df_instances[['instance', 'number_of_movements']] 
# only keep the instace 151-200
df_instances = df_instances.sort_values(by=['instance'])
df_instances = df_instances[150:200]

# merge the instances with the SA&ML data
df_ml = pd.merge(df_ml, df_instances, on='instance')
df_tabu = pd.merge(df_tabu, df_instances, on='instance')

# to excel for checking
df_ml.to_excel('results/TABU/ML-Results/check.xlsx')
df_tabu.to_excel('results/TABU/check.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'results/TABU/ML-Results/ML151-200x10-redo-c.xlsx'

In [21]:
# calculate the amouunt of valid solutions
print('ML valid solutions: ', df_ml['valid_solution-ML'].sum())
print('TABU valid solutions: ', df_tabu['valid_solution-TABU'].sum())

# calculate the average delay
df_results = pd.DataFrame()
df_results['avg_movements_reached-ML'] = df_ml['movements_reached-ML']/df_ml['number_of_movements']
df_results['avg_movements_reached-TABU'] = df_tabu['movements_reached-TABU']/df_tabu['number_of_movements']
average_ML = df_results['avg_movements_reached-ML'].sum() / len(df_results)
average_TABU = df_results['avg_movements_reached-TABU'].sum() / len(df_results)
print('ML average movements reached: ', average_ML)
print('TABU average movements reached: ', average_TABU)

print(len(df_ml))
print(len(df_tabu))


ML valid solutions:  100
TABU valid solutions:  83
ML average movements reached:  0.7624987157765534
TABU average movements reached:  0.7492172379286461
500
500


In [22]:
# add the solved column from ml to tabu
df_tabu['valid_solution-ML'] = df_ml['valid_solution-ML']
print(df_tabu['valid_solution-ML'])
df_ml['valid_solution-TABU'] = df_tabu['valid_solution-TABU']

# calculate the average on the solved instances
df_ml_solved = df_ml[df_ml['valid_solution-ML'] == 1]
df_tabu_solved = df_tabu[df_tabu['valid_solution-TABU'] == 1]

# now only keep if the instance is solved in both ML and TABU
df_ml_solved = df_ml_solved[df_ml_solved['valid_solution-TABU'] == 1]
df_tabu_solved = df_tabu_solved[df_tabu_solved['valid_solution-ML'] == 1]

# calculate the amount of valid solutions
print('ML valid solutions: ', df_ml_solved['valid_solution-ML'].sum())
print('TABU valid solutions: ', df_tabu_solved['valid_solution-TABU'].sum())

# calculate the average wait time
avg_delay_ML = df_ml_solved['average_delay-ML'].sum() / len(df_ml_solved)
avg_delay_TABU = df_tabu_solved['average_delay-TABU'].sum() / len(df_tabu_solved)
print('ML average delay: ', decimal_to_time(avg_delay_ML))
print('TABU average delay: ', decimal_to_time(avg_delay_TABU))

# calculate the average median delay for ML and TABU
avg_median_ML = df_ml_solved['median_delay-ML'].sum() / len(df_ml_solved)
avg_median_TABU = df_tabu_solved['median_delay-TABU'].sum() / len(df_tabu_solved)
print('ML average median delay: ', decimal_to_time(avg_median_ML))
print('TABU average median delay: ', decimal_to_time(avg_median_TABU))

# calculate the average obj val for ML and TABU
avg_obj_ML = df_ml_solved['obj_val-ML'].sum() / len(df_ml_solved)
avg_obj_TABU = df_tabu_solved['obj_val-TABU'].sum() / len(df_tabu_solved)
print('ML average obj val: ', avg_obj_ML)
print('TABU average obj val: ', avg_obj_TABU)

# calcute the standard deviation for ML and TABU on the average delay
std_ML = df_ml_solved['average_delay-ML'].std()
std_TABU = df_tabu_solved['average_delay-TABU'].std()
print('ML standard deviation: ', decimal_to_time(std_ML))
print('TABU standard deviation: ', decimal_to_time(std_TABU))


0      0
1      0
2      0
3      0
4      0
      ..
495    0
496    0
497    0
498    0
499    0
Name: valid_solution-ML, Length: 500, dtype: int64
ML valid solutions:  64
TABU valid solutions:  64
ML average delay:  0:37
TABU average delay:  0:37
ML average median delay:  0:27
TABU average median delay:  0:29
ML average obj val:  63.20703125000067
TABU average obj val:  60.403645833334004
ML standard deviation:  0:21
TABU standard deviation:  0:20


In [23]:
# compare on instances that both ML and TABU solved
df_tabu['valid_solution-ML'] = df_ml['valid_solution-ML']
df_ml['valid_solution-TABU'] = df_tabu['valid_solution-TABU']
df_ml_solved = df_ml[df_ml['valid_solution-ML'] == 1]
df_tabu_solved = df_tabu[df_tabu['valid_solution-TABU'] == 1]
df_ml_solved = df_ml_solved[df_ml_solved['valid_solution-TABU'] == 1]
df_tabu_solved = df_tabu_solved[df_tabu_solved['valid_solution-ML'] == 1]

# calculate the amount of valid solutions for SA and ML
print('ML valid solutions: ', df_ml_solved['valid_solution-ML'].sum())
print('TABU valid solutions: ', df_tabu_solved['valid_solution-TABU'].sum())

obj_val_diff = df_ml_solved['obj_val-ML'] - df_tabu_solved['obj_val-TABU']
avg_diff = df_ml_solved['average_delay-ML'] - df_tabu_solved['average_delay-TABU']
median_diff = df_ml_solved['median_delay-ML'] - df_tabu_solved['median_delay-TABU']
print('avg delay difference: ', avg_diff.mean())
print('avg delay difference std: ', avg_diff.std())
print('median delay difference: ', median_diff.mean())
print('median delay difference std: ', median_diff.std())
print('obj val difference: ', obj_val_diff.mean())
print('obj val difference std: ', obj_val_diff.std())






ML valid solutions:  64
TABU valid solutions:  64
avg delay difference:  0.0004709102208134159
avg delay difference std:  0.44180371792002776
median delay difference:  -0.026041666666666786
median delay difference std:  0.4685607348652845
obj val difference:  2.803385416666666
obj val difference std:  54.166158661003294


In [29]:
# now for the reasonable comparison
import pandas as pd
df_tabu = pd.read_excel('results/TABU/output_151-200ex10-REAS.xlsx')
df_tabu = df_tabu.rename(columns={'number of movements': 'movements_reached-TABU', 'median delay': 'median_delay-TABU', 'average delay': 'average_delay-TABU', 'obj_val': 'obj_val-TABU', 'solution_found': 'valid_solution-TABU', 'predicted_delay': 'predicted_delay-TABU', 'tabu_list_size': 'tabu_list_size-TABU', 'number_of_tweaks': 'number_of_tweaks-TABU', 'affected_movements': 'affected_movements-TABU',})

print(df_tabu.columns)
# load in the ml data
df_ml1 = pd.read_excel('results/TABU/ML-Results/ML151-200x10-REAS-redo-c.xlsx')
# df_ml2 = pd.read_excel('results/TABU/ML-Results/ML151-200x10-REAS-redo-c_ctd1.xlsx')

df_ml = pd.concat([df_ml1, df_ml2])
df_ml = df_ml.rename(columns={'number of movements': 'movements_reached-ML', 'median delay': 'median_delay-ML', 'average delay': 'average_delay-ML', 'obj_val': 'obj_val-ML', 'solution_found': 'valid_solution-ML', 'predicted_delay': 'predicted_delay-ML','tabu_list_size': 'tabu_list_size-ML', 'number_of_tweaks': 'number_of_tweaks-ML', 'affected_movements': 'affected_movements-ML',})
print(df_ml.columns)
print(len(df_ml))
print(len(df_tabu))

# to excel for checking
df_ml.to_excel('results/TABU/ML-Results/check.xlsx')
df_tabu.to_excel('results/TABU/check.xlsx')

Index(['instance', 'movements_reached-TABU', 'median_delay-TABU',
       'average_delay-TABU', 'obj_val-TABU', 'tabu_list_size-TABU',
       'number_of_tweaks-TABU', 'affected_movements-TABU', 'epochs',
       'time_interval', 'vessel_time_window', 'valid_solution-TABU'],
      dtype='object')
Index(['instance', 'movements_reached-ML', 'median_delay-ML',
       'average_delay-ML', 'obj_val-ML', 'tabu_list_size-ML',
       'number_of_tweaks-ML', 'affected_movements-ML', 'epochs',
       'time_interval', 'vessel_time_window', 'valid_solution-ML',
       'predicted_delay-ML'],
      dtype='object')
500
500


In [30]:
print(df_ml.index.is_unique)

# drop the index that is not unique
df_ml = df_ml.reset_index(drop=True)
print(df_ml.index.is_unique)
df_tabu.to_excel('results/TABU/check.xlsx')
print(df_ml.index.is_unique)

False
True
True


In [31]:
# compare on instances that both ML and TABU solved
df_tabu['valid_solution-ML'] = df_ml['valid_solution-ML']
df_ml['valid_solution-TABU'] = df_tabu['valid_solution-TABU']
df_ml_solved = df_ml[df_ml['valid_solution-ML'] == 1]
df_tabu_solved = df_tabu[df_tabu['valid_solution-TABU'] == 1]
df_ml_solved = df_ml_solved[df_ml_solved['valid_solution-TABU'] == 1]
df_tabu_solved = df_tabu_solved[df_tabu_solved['valid_solution-ML'] == 1]

# calculate the amount of valid solutions for SA and ML
print('ML valid solutions: ', df_ml_solved['valid_solution-ML'].sum())
print('TABU valid solutions: ', df_tabu_solved['valid_solution-TABU'].sum())

obj_val_diff = df_ml_solved['obj_val-ML'] - df_tabu_solved['obj_val-TABU']
avg_diff = df_ml_solved['average_delay-ML'] - df_tabu_solved['average_delay-TABU']
median_diff = df_ml_solved['median_delay-ML'] - df_tabu_solved['median_delay-TABU']
print('avg delay difference: ', avg_diff.mean())
print('avg delay difference std: ', avg_diff.std())
print('median delay difference: ', median_diff.mean())
print('median delay difference std: ', median_diff.std())
print('obj val difference: ', obj_val_diff.mean())
print('obj val difference std: ', obj_val_diff.std())

ML valid solutions:  49
TABU valid solutions:  49
avg delay difference:  0.14764612737325505
avg delay difference std:  0.4348458318046567
median delay difference:  0.09693877551020387
median delay difference std:  0.44951972144697444
obj val difference:  2.0238095238095974
obj val difference std:  56.50450397895224


In [14]:
# THE COMPLETELY RANDOM MODEL
df_ml = pd.read_excel('results/TABU/ML-Results/ML151-200x10-UNREAS-redo-c.xlsx')

df_ml = df_ml.rename(columns={'number of movements': 'movements_reached-ML', 'median delay': 'median_delay-ML', 'average delay': 'average_delay-ML', 'obj_val': 'obj_val-ML', 'solution_found': 'valid_solution-ML', 'predicted_delay': 'predicted_delay-ML','tabu_list_size': 'tabu_list_size-ML', 'number_of_tweaks': 'number_of_tweaks-ML', 'affected_movements': 'affected_movements-ML',})

df_tabu = pd.read_excel('results/TABU/output_151-200ex10-UNREAS.xlsx')
df_tabu = df_tabu.rename(columns={'number of movements': 'movements_reached-TABU', 'median delay': 'median_delay-TABU', 'average delay': 'average_delay-TABU', 'obj_val': 'obj_val-TABU', 'solution_found': 'valid_solution-TABU', 'predicted_delay': 'predicted_delay-TABU', 'tabu_list_size': 'tabu_list_size-TABU', 'number_of_tweaks': 'number_of_tweaks-TABU', 'affected_movements': 'affected_movements-TABU',})




In [15]:
# compare on instances that both ML and TABU solved
df_tabu['valid_solution-ML'] = df_ml['valid_solution-ML']
df_ml['valid_solution-TABU'] = df_tabu['valid_solution-TABU']
df_ml_solved = df_ml[df_ml['valid_solution-ML'] == 1]
df_tabu_solved = df_tabu[df_tabu['valid_solution-TABU'] == 1]
df_ml_solved = df_ml_solved[df_ml_solved['valid_solution-TABU'] == 1]
df_tabu_solved = df_tabu_solved[df_tabu_solved['valid_solution-ML'] == 1]

# calculate the amount of valid solutions for SA and ML
print('ML valid solutions: ', df_ml_solved['valid_solution-ML'].sum())
print('TABU valid solutions: ', df_tabu_solved['valid_solution-TABU'].sum())

obj_val_diff = df_ml_solved['obj_val-ML'] - df_tabu_solved['obj_val-TABU']
avg_diff = df_ml_solved['average_delay-ML'] - df_tabu_solved['average_delay-TABU']
median_diff = df_ml_solved['median_delay-ML'] - df_tabu_solved['median_delay-TABU']
print('avg delay difference: ', avg_diff.mean())
print('avg delay difference std: ', avg_diff.std())
print('median delay difference: ', median_diff.mean())
print('median delay difference std: ', median_diff.std())
print('obj val difference: ', obj_val_diff.mean())
print('obj val difference std: ', obj_val_diff.std())

ML valid solutions:  12
TABU valid solutions:  12
avg delay difference:  -0.11883090248770072
avg delay difference std:  0.34525206241079476
median delay difference:  -0.138888888888889
median delay difference std:  0.3761905903519652
obj val difference:  -11.770833333333336
obj val difference std:  30.076777280248248
